In [1]:
from app.Transformer_Classifier import Transformer_Classifier 
from app.BQUtility import BQUtility
from app.Data_Loader import Data_Loader
from app.TextRank_Extractor import TextRank_Extractor
from app.Keyword_Classifier import Keyword_Classifier

class Data_ETL_Pipeline:
    def __init__(self) -> None:
        pass

    dbutil = BQUtility()
    data_load = Data_Loader()
    textrank = TextRank_Extractor()
    key_classifier = Keyword_Classifier()
    class_service = Transformer_Classifier()

    def create_dataset(self):
        print("create_database():")
        self.dbutil.create_database() 

    def load_seed_training_data(self):
        print("import_seed_data():")
        self.data_load.import_seed_data()

        print("extract_keyword_seed_data():")
        self.textrank.extract_keyword_seed_data()

        print("load_seed_to_training_data():")
        self.data_load.load_seed_to_training_data() 
    
    def load_contract_data(self):
        print("self.data_load.import_reports_data()")
        self.data_load.import_reports_contract_data()

        print("data_load.import_cuad_contract_data():")
        #self.data_load.import_cuad_contract_data()

    def process_keyword_model(self):
        print("key_classifier.prepare_training_data():")
        self.key_classifier.prepare_training_data()

        print("key_classifier.train_model():")
        self.key_classifier.train_model()

        print("key_classifier.evaluate_model():")
        self.key_classifier.evaluate_model()

        print("key_classifier.process_contract_data():")
        self.key_classifier.process_contract_data()

    def process_transformer_model(self):
        print("prepare_train_dataset():")
        train_hg, valid_hg = self.class_service.prepare_train_dataset()

        print("training():")
        self.class_service.training(train_hg, valid_hg)    

        print("evaluate_contract_training_data():")
        self.class_service.process_contract_training_data_eval()

    def evalue_results(self):
        print ("Keyword Classifier Accuracy: ")
        self.key_classifier.evaluate_model() 
        
        print ("Transformer Classifier Accuracy: ")
        self.class_service.evalute_model()

if __name__ == '__main__': 
    dbloader = Data_ETL_Pipeline()
    
    dbloader.create_dataset()
    dbloader.seed_training_dataset()
    dbloader.process_training_dataset()
    dbloader.model_evaluation()
    dbloader.evalue_results()


/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-12 11:32:48.838494: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


self.data_load.import_reports_data()
import_contract_data():
key_classifier.prepare_training_data():
SELECT * from genuine-wording-362504.lca_db.seed_data
key_classifier.train_model():
key_classifier.evaluate_model():
Classification Report : 
                          precision    recall  f1-score   support

 Contingent liabilities       1.00      0.63      0.77        19
    Current liabilities       0.96      1.00      0.98        54
Non-current liabilities       0.92      1.00      0.96        56

               accuracy                           0.95       129
              macro avg       0.96      0.88      0.90       129
           weighted avg       0.95      0.95      0.94       129

Confusion Matrix: 
 [[12  2  5]
 [ 0 54  0]
 [ 0  0 56]]
Accuracy Score: 
 94.57364341085271 %
key_classifier.process_contract_data():
SELECT * from genuine-wording-362504.lca_db.contract_data Limit 50
Filename: IBM_Annual_Report_2002.txt
Sentences :  Current assets
$Ç41,652
$Ç42,461
Current liabi

AttributeError: 'Transformer_Classifier' object has no attribute 'dbutil'